In [1]:
from library import table, append_table, historize, get_teams, get_competitions
import pandas as pd

### Leagues

In [2]:
leagues_history = table("league")

league_data = get_competitions()
leagues =  pd.DataFrame(league_data, columns =  ['name', 'link'])
leagues = pd.merge(leagues.assign(joined =  1), leagues_history, on =  ['name', 'link'], how = 'outer')
leagues = leagues[leagues.joined != 1]

if len(leagues > 0):
    print(f"Committing {len(leagues)} records")
    leagues = leagues.drop("joined", axis = 1)
    append_table(leagues, league)
else:
    print("League data up-to-date")

No league updates today!


### Clubs

In [3]:
leagues = table("league")
league_urls = leagues.link.unique()

In [ ]:
teams_df = pd.DataFrame()

#Scrape club data
for url in league_urls:
    teams_df = pd.concat([teams_df, get_teams(url)])

#Prepare df
teams_df = teams_df.dropna(subset = 'Club.1')
selection = ['Club.1', 'link']
teams_df = teams_df[selection].drop_duplicates().rename({"Club.1" : "club"}, axis = 1)

#Add hash keys and effective dates
teams_df = historize(teams_df)

#Filter only new data
current_clubs = table("club")
updates = teams_df[~teams_df.hash_key.isin(current_clubs.hash_key.unique())]

#Commit changes
if len(updates) > 0:
    print(f"Adding {len(updates)} new club records")
    append_table(updates ,'club')
else:
    print("Club data up-to-date")

https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1
https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1
https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1
https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1
https://www.transfermarkt.com/liga-portugal/startseite/wettbewerb/PO1
https://www.transfermarkt.com/eredivisie/startseite/wettbewerb/NL1
https://www.transfermarkt.com/jupiler-pro-league/startseite/wettbewerb/BE1
https://www.transfermarkt.com/super-liga-srbije/startseite/wettbewerb/SER1
https://www.transfermarkt.com/super-league-1/startseite/wettbewerb/GR1
https://www.transfermarkt.com/allsvenskan/startseite/wettbewerb/SE1
